The Scenario: You are an internal engineer at a Hollywood studio. Executives keep asking you: "Did this movie make money? What was the ROI?" Your Job: Build an Agent that can look up private financial data (which Gemini doesn't know) and calculate the profit.

In [ ]:
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor
import os


os.environ["GOOGLE_API_KEY"] = "apikey"

llm = Gemini(model="gemini-2.5-flash")



In [41]:
def get_movie_financials(movie_name:str)-> dict:
    
    """
    This tool accepts movie name as input and searches the database for a particular movie and then looks up the financial data.
    It also states if the movie was profitable or in loss and it specific the exact number.

    
    Args: The name of the movie , it should be particular / specific and it should be in lower case. example: "interstellar" , "oppenheimer"

    Returns:
    A dictionary with financial data(profit or loss)
    Success: {"status": "success", "loss": -53,67,00,000 }
    Error: {"status": "error", "error_message": "movie not found"}
    """
    movie_db = {
    "interstellar": {"budget": 165000000, "box_office": 701700000},
    "tenet": {"budget": 200000000, "box_office": 363700000},
    "oppenheimer": {"budget": 100000000, "box_office": 952000000},
    }
    
    movie = movie_db.get(movie_name.lower())

    if movie is None:
        return {"status": "error", "error_message": "movie not found"}
        
    budget = movie["budget"]
    box_office = movie["box_office"]

    
    
    if box_office > budget:
        profit = box_office - budget
        return {"status":"success","profit": f"${profit: ,}"}
    else:
        loss = budget - box_office
        return {"status": "success", "loss": f"${loss: ,}"}




In [42]:
Movie_agent = LlmAgent(
    name="movie_agent",
    model = llm,
    instruction= "You are a financial data calculator/analyser for a hollywood company.Your main task is to retrieve the data(movie name) and search for the movie in the database and once its found , search for its financial data , return the profit or loss that the movie made and the output should be in readable format.If movie's budget is more than box_office then it should be labelled as loss and if box_office is greater than budget then thats a profit",
    tools = [get_movie_financials]


)
currency_runner = InMemoryRunner(agent=Movie_agent)
_ = await currency_runner.run_debug(
    "I want to know  oppenheimer's financial data"
)



        


 ### Created new session: debug_session_id

User > I want to know  oppenheimer's financial data


movie_agent > "Oppenheimer" made a profit of $852,000,000.
